In [ ]:
import sys
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

# file locations
BASEPATH = {
    'darwin': '/Users/patrick/Dropbox/datascienceud/vis/raw',
    'linux': '/home/patrick/Dropbox/datascienceud/vis/raw',
}
basepath = BASEPATH.get(sys.platform)

csvfilename = os.path.join(basepath, 'fligth_delay_cause_monthly_2003_2017.csv')
airportscsv = os.path.join(basepath, 'master_airports.csv')

# read raw data
delays = pd.read_csv(csvfilename)
delays.columns = [col.strip() for col in delays.columns]

airports = pd.read_csv(airportscsv)
airports.columns = [c.lower() for c in airports.columns]

# utilities
c = lambda v: v.split(',')

# get unique data
airports = airports.drop_duplicates(c('airport'))
assert len(airports.groupby('airport').filter(lambda v: len(v) > 1)) == 0, "there are duplicates"
airports.columns

# merge delays and airport fips
delays = delays.merge(airports[c('airport,airport_state_code,airport_state_fips')], on=c('airport'), how='left')
delays['arr_del15_pct'] = delays.arr_del15 / delays.arr_flights
delays.rename(columns=dict(airport_state_fips='fips',
                           airport_state_code='state'), inplace=True)
delays['fips'] = delays.fips.dropna().apply(lambda v: 'US{:02d}'.format(int(v)))

# calculate percentage of flights per cause
for ctcol in [col for col in delays.columns if col.endswith('_ct')]:
    delays['{}_pct'.format(ctcol)] = delays[ctcol] / delays['arr_del15']
    
# average delay
delays['arr_del15_avg'] = delays.arr_delay / delays.arr_flights

# prepare delays by cause csv
groupcol = ['year']
ctpctcols = [col for col in delays.columns if col.endswith('_ct_pct')]
dfx = delays[groupcol + ctpctcols].groupby(groupcol).median()
dfx.plot(by=groupcol)
dfx.reset_index().to_csv('data/delays_cause_year.csv')
dfx = dfx.reset_index()
dfx = dfx.melt(id_vars=['year'], var_name='name')
dfx.to_csv('data/delays_cause_year.csv')

# prepare by airport csv
cols = c('year,month,carrier,carrier_name,airport_name,fips,state,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct')
dfx = delays[cols]
dfx = dfx.groupby(c('year,state,fips')).sum().reset_index()
dfx = dfx.groupby(c('state,fips')).mean().reset_index()
dfx.to_csv('data/airport_causes.csv', float_format='%.3f', quoting=csv.QUOTE_NONNUMERIC)
dfx.sort_values('arr_del15', ascending=False)

# cause by carrier
for v in ['arr_del15', 'carrier_ct_pct', 'nas_ct_pct', 'late_aircraft_ct_pct']:
    plt.figure()
    plt.suptitle(v)
    total = delays[v].sum()
    causeby = delays.groupby(['carrier']).agg({v: lambda v: v.sum() / total})
    #causeby = causeby.nlargest(10, v)
    causeby = causeby[v].sort_values()
    causeby.plot.bar()
    break
dfx = causeby.reset_index().sort_values('arr_del15').to_csv('data/causeby_carrier.csv')